<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/Molecular_Modelling_Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##Setting up a Simulation

This set-up uses [CHARMM36m](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs).

Change the settings below and click **Runtime → Run all**.

You will be prompted to enter a RCSB [PDB](http://rcsb.org) ID structure file to set-up for simulation.

In [111]:
#@title Initialisation
%%capture
!pip install py3dmol
!apt-get update && apt-get upgrade -y && apt-get install -y gzip pymol dssp
File_Location = "RCSB PDB" #@param ["Upload","RCSB PDB","AFDB","Swiss-Model","Sequence"]
PDB_ID = "1AKI" #@param {type:"string"}


In [112]:
#@title Get PDB coordinate file
import os
import sys
import requests
import py3Dmol
from google.colab import files

sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

if File_Location == "RCSB PDB":
    name = str(PDB_ID.lower())
    os.system('wget http://www.rcsb.org/pdb/files/' + name + '.pdb1.gz')
    os.system('gunzip ' + name + '.pdb1.gz')
    filename = name + '.pdb'
    os.system('egrep -v "MODEL|ENDMDL|HOH|SOL" '+ name + '.pdb1 >'+ filename)
    name = os.path.splitext(filename)[0]
    working_dir = '/content/' + name + '/'
    files = os.listdir(working_dir)
    for file in files:
      file_path = os.path.join(working_dir, file)
      if os.path.isfile(file_path):
        os.remove(file_path)
    os.makedirs(working_dir, exist_ok=True)
    os.rename(filename, working_dir + name + '.pdb')
    os.chdir(working_dir)
    py3dmol_view(filename, working_dir)

def py3dmol_view(pdbfilename, working_dir):
    mview = py3Dmol.view(width=800, height=400)
    with open(working_dir + pdbfilename, "r") as f:
        mol1 = f.read()
    mview.addModel(mol1, "pdb")
    mview.setStyle({"cartoon": {"color": "spectrum"}})
    mview.setStyle({'atom': 'NA'}, {'sphere': {"color": "blue"}})
    mview.setStyle({'atom': 'CL'}, {'sphere': {"color": "yellow"}})
    mview.setStyle({'resn': 'SOL'}, {'sphere': {"radius": 0.3,"color": "red"}})
    mview.setBackgroundColor("0xffffff")
    mview.zoomTo()
    mview.addUnitCell()
    mview.show()

def energy_minimize(filename, working_dir):
    os.system(f"pdb2pqr30 --ff CHARMM --keep-chain {working_dir + filename} pqr.pdb")

    pdb2gmx(f="pqr.pdb", ignh=True, ff="charmm27", water="tip3p", o="conf.pdb")
    editconf(f="conf.pdb", d=2, c=True, o="conf.pdb")

    with open("em.mdp", "w") as em:
        em.write("integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001")

    grompp(f="em.mdp", maxwarn=5, o="em", c="conf.pdb")
    mdrun(deffnm="em", c="clean.pdb")

    trjconv(f="clean.pdb", o=working_dir + f"fixed-{filename}", s="em.tpr", input=("system"))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [113]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/content/gromacs"):
  !pip install pdb2pqr GromacsWrapper MDAnalysis wget
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil
import subprocess
import wget
import pandas as pd
from gromacs import grompp, editconf, make_ndx, trjconv, confrms, pdb2gmx, mdrun, solvate, genion

In [ ]:
#@title Set-up Periodic Unit Cell & Energy Minimise Input Structure
os.chdir(working_dir)
energy_minimize(filename, working_dir)
py3dmol_view(f"fixed-{filename}", working_dir)

In [ ]:
#@title Fill the box with Water
os.chdir(working_dir)
solvate(cp=f"fixed-{filename}", cs=True, o=f"solated-{filename}", p="topol.top")
py3dmol_view(f"solated-{filename}", working_dir)

In [ ]:
#@title Neutralise the system
os.chdir(working_dir)
grompp(f='em.mdp', c=f"solated-{filename}", o=f"counter.tpr")
genion(s='counter.tpr', o="System.pdb", p="topol.top", neutral=True, conc=0.2,input=str(13),backup=False)
py3dmol_view(f'System.pdb', working_dir)